In [1]:
import json
import os
import random
from typing import List, Dict
import uuid

from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel, Extra, Field
import tiktoken

from config import settings, app_settings

In [2]:
from src.simschat.models import CharacterSpecification, create_dynamic_enum

In [3]:
openai_client = OpenAI(api_key=settings.openai_api_key)
tokenizer = tiktoken.encoding_for_model("gpt-4o")

# 2. Scene Construction

## 2-1. Sample Characters
* select 2 characters from collection

In [4]:
with open("simschat/character_collection.json", "r") as f:
    character_collection = json.load(f)

character_ids = list(character_collection["source"].keys())

In [5]:
# selected_character_ids = random.sample(character_ids, 2)

# character1_id = selected_character_ids[0]
# character2_id = selected_character_ids[1]

# fix characters for testing
character1_id="35f0c56f-263d-42df-846c-e1833d8ca0ab"
character2_id="00d66087-9b3b-46da-bd74-bf45cbe81d3c"

print(character1_id, character_collection["source"][character1_id])
print(character2_id, character_collection["source"][character2_id])

35f0c56f-263d-42df-846c-e1833d8ca0ab wiki_Zephyr Orion.txt
00d66087-9b3b-46da-bd74-bf45cbe81d3c wiki_Vivienne LaRoux.txt


In [6]:
# Load Characters
with open(f"simschat/characters/{character1_id}.json", "r") as f:
    character1_spec = CharacterSpecification(**json.load(f))
    
with open(f"simschat/characters/{character2_id}.json", "r") as f:
    character2_spec = CharacterSpecification(**json.load(f))

In [7]:
class Character(BaseModel):
    uid: str
    spec: CharacterSpecification

In [8]:
character1 = Character(
    uid=character1_id,
    spec=character1_spec
)

character2 = Character(
    uid=character2_id,
    spec=character2_spec
)

In [9]:
print(character1.model_dump_json(indent=4))

{
    "uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "spec": {
        "name": "Zephyr Orion",
        "gender": "male",
        "age": 28,
        "dialogue_tone": "playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.",
        "career": "Astronaut",
        "personality_traits": [
            {
                "trait": "Goofball",
                "description": "Enjoys joking and making others laugh, bringing a playful spirit to social situations."
            },
            {
                "trait": "Materialistic",
                "description": "Loves acquiring new possessions and often leans towards bragging about them."
            },
            {
                "trait": "Outgoing",
                "description": "Flourishes in social situations and enjoys being around people."
            },
            {
                "trait": "Gloomy",
                "descrip

## 2-1. Generate initial trope

In [10]:
TROPE_INIT_INSTRUCTION = '''Given the following 2 character specifications write a story trope that is suitable for the interaction of these characters.

[Character1]
{character1}
[Character2]
{character2}

Return in the following json format
{{"trope": str}}

Rules:
* trope must be detailed description in minimum 3 sentences.
* trope must focus on character behavioral, emotional interaction and less about the surrounding settings (environment) of the characters
* trope should describe a single small scene, keep the scope small
* mention the character's gender, age, personality, living condition that are only relevant to the trope'''

class InitTropeResult(BaseModel):
    trope: str
    class Config:
        extra = Extra.forbid

def generate_initial_trope(
    character1: CharacterSpecification,
    character2: CharacterSpecification,
):
    instruction = TROPE_INIT_INSTRUCTION.format(
        character1=character1.model_dump_json(),
        character2=character2.model_dump_json(),
    )
    # print(instruction)
    messages = [
        {"role": "user", "content": instruction}
    ]
    decode_params = {"temperature": 0.95}

    response = openai_client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=messages,
        response_format=InitTropeResult,
        **decode_params,
    )
    return response.choices[0].message.parsed.trope
    

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/1748596206.py:20: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [11]:
# initial_trope=generate_initial_trope(
#     character1, character2
# )
initial_trope="In the glamorous hustle and bustle of a metropolitan fashion gala, **Zephyr Orion**, a 28-year-old social magnet with an astronaut's bravado, finds himself drawn to the assertive allure of **Vivienne LaRoux**, a renowned 28-year-old style influencer. Standing out in the crowd with his easy-going charm, Zephyr playfully engages Vivienne in a conversation, his jocular tone a stark contrast to her sharply assertive style. She, poised and confident, initially dismisses Zephyr's witty banter with an elegant roll of the eyes, but his relentless charisma gradually cracks her facade, revealing a flicker of intrigued amusement. Despite Vivienne's mean streak occasionally surfacing, the evening unfolds into a dynamic exchange, Zephyr's warmth melting Vivienne's icy exterior as they navigate through complex social dance, caught between playful banter and subtle challenges to each other's worldviews."

In [12]:
print(initial_trope)

In the glamorous hustle and bustle of a metropolitan fashion gala, **Zephyr Orion**, a 28-year-old social magnet with an astronaut's bravado, finds himself drawn to the assertive allure of **Vivienne LaRoux**, a renowned 28-year-old style influencer. Standing out in the crowd with his easy-going charm, Zephyr playfully engages Vivienne in a conversation, his jocular tone a stark contrast to her sharply assertive style. She, poised and confident, initially dismisses Zephyr's witty banter with an elegant roll of the eyes, but his relentless charisma gradually cracks her facade, revealing a flicker of intrigued amusement. Despite Vivienne's mean streak occasionally surfacing, the evening unfolds into a dynamic exchange, Zephyr's warmth melting Vivienne's icy exterior as they navigate through complex social dance, caught between playful banter and subtle challenges to each other's worldviews.


## 2-2. Retrieve similar tropes

In [13]:
from qdrant_client import QdrantClient
from llama_index.core import (
	Document,
	SimpleDirectoryReader, VectorStoreIndex,
	StorageContext,
	Settings,
	QueryBundle
)

from llama_index.embeddings.text_embeddings_inference import (
    TextEmbeddingsInference,
)

from src.tropes.retriever import TropeRetriever

In [14]:
embedder = TextEmbeddingsInference(
    settings.embedding_model,
    base_url=settings.embedding_base_url,
    auth_token=settings.embedding_api_key
)
Settings.embed_model = embedder
client = QdrantClient(host="localhost", port=app_settings.qdrant_port)

retriever = TropeRetriever(client)

In [15]:
retrieved_tropes = retriever.retrieve(initial_trope)
print(len(retrieved_tropes))

5


In [16]:
for trope in retrieved_tropes:
    print(trope)

trope_id='t13173' name='LoonyFriendsImproveYourPersonality' explanation='A stiff character learns and grows from unwanted interaction with annoying and eccentric people.\nThe focus character is a fairly stiff guy (rarely a gal), may be the Only Sane Man, The Comically Serious or a Jerkass of the "stick up his rear" type, and often comes across as an Ineffectual Loner. He may not be content with his life, but it\'s stable, and he probably has a long-term plan for fixing what he thinks is wrong, if he can just get the right breaks. Instead, he is dragged into wacky hijinks by the other characters against his will, making a mess of his life. But implicitly or explicitly, the goal in the story is to make him a better person by putting him through "horrible" experiences - or at least ones he considers horrible, in order to show him the brighter side of things.\nMay involve a Manic Pixie Dream Girl, but in most examples, it\'s an entire cast imposing on the stiff guy\'s time, money and patie

## 2-4. Refine

In [17]:
def format_trope(trope):
    return "{{\"name\": {name}, \"explanation\": {explanation}}}".format(
        name=trope.name, explanation=trope.explanation
    )
    
class SceneState(BaseModel):
    location: str
    setting: str
    explanation: str
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/1847138306.py:12: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [18]:
TROPE_REFINE_INSTRUCTION = '''You are a storyteller which is coming up with a suitable scene using the given information.
# Given Informations
You will be given the following informations
* 2 Character descriptions
* A draft version of the story trope involving these 2 characters
* Reference story tropes that can be used to refine the draft

Revise the draft trope by taking into considerations the Reference Tropes.
* Only utilize the reference tropes if they are applicable to the given characters

## Characters
Character1:
{character1}

Character2:
{character2}

## Draft Trope
{initial_trope}

## Reference Tropes
{reference_tropes}

# Refine Process:
Refine the draft trope by writing the following informations
* 'revised_trope': 
* 'utilized_reference_tropes':
* 'scene': explanation of the scene with background informations including location/setting

Return in the following json format
{{
    "revised_trope": str,
    "utilized_reference_tropes": List[str],
    "scene": {{
        "location": str,
        "setting": str,
        "explanation": str
    }}
}}
* only return the name of the referenced tropes in utilized_reference_tropes

Rules:
* trope must be detailed description in minimum 3 sentences.
* trope must focus on character behavioral, emotional interaction and less about the surrounding settings (environment) of the characters
* trope should describe a single small scene, keep the scope small
* mention the character's gender, age, personality, living condition that are only relevant to the trope
'''

In [19]:
refine_instruction = TROPE_REFINE_INSTRUCTION.format(
    character1=character1.spec.model_dump_json(),
    character2=character2.spec.model_dump_json(),
    initial_trope=initial_trope,
    reference_tropes="\n".join([format_trope(x) for x in retrieved_tropes])
)
# print(refine_instruction)

In [20]:
class RefineTropeOutput(BaseModel):
    revised_trope: str
    utilized_reference_tropes: List[str]
    scene: SceneState
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/2621776176.py:7: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [21]:
messages = [
    {"role": "user", "content": refine_instruction}
]
decode_params = {"temperature": 0.95}

response = openai_client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=RefineTropeOutput,
    **decode_params,
)
initialized_scene = response.choices[0].message.parsed

In [22]:
print(initialized_scene.model_dump_json(indent=4))

{
    "revised_trope": "Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.",
    "utilized_reference_tropes": [
        "Defrosting Ice Queen",
        "SavvyGuyEnergeticGirl"
    ],
    "scene": {
        "location": "The Metropolitan Fashion Gala",
        "setting": "An opulent event hall adorned with sparkling chandeliers and cutting-edge fas

# 3. Character state initialization

In [40]:
## Emotion
emotion_descriptions = {
    "joy": "Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance social connections and overall resilience by radiating positivity",
    "trust": "Trust is the reassuring feeling of confidence and security in another person or situation. It builds from consistent, reliable interactions and underpins strong, supportive relationships. This emotion fosters cooperation and reduces anxiety by creating a sense of safety",
    "fear": "Fear is an instinctive response to perceived threats that activates our fight-or-flight mechanism. It heightens awareness and prepares our body to respond quickly to danger, making it essential for survival. Despite its discomfort, fear is a crucial signal that prompts protective action and risk assessment",
    "surprise": "Surprise occurs when we encounter the unexpected, momentarily halting our regular thought process. This emotion can be positive, neutral, or even negative, depending on the context, and often sparks curiosity about what comes next. Its brief nature helps redirect our focus and encourages adaptive responses to new situations",
    "sadness": "Sadness is a deep, reflective emotion that often emerges from loss, disappointment, or unmet expectations. It can lead to introspection and a desire for support as we navigate feelings of grief or dejection. Although challenging, sadness can also foster empathy and pave the way for emotional healing and growth",
    "disgust": "Disgust is an aversive emotion that signals rejection toward something perceived as harmful, unclean, or morally offensive. It serves as a protective mechanism, prompting us to avoid substances or situations that might be dangerous. This emotion plays a vital role in maintaining both physical health and ethical boundaries",
    "anger": "Anger arises when we perceive injustice, frustration, or a threat to our well-being, often urging us to act in response. It can manifest as physical tension and heightened energy, signaling that something in our environment needs to change. When managed effectively, anger can motivate constructive action and help assert personal boundaries",
    "anticipation": "Anticipation is the forward-looking emotion characterized by a mix of excitement and apprehension about future events. It motivates preparation and planning while balancing hope with cautious vigilance. This emotion bridges the gap between our present state and the potential for positive outcomes in the future",
}

EmotionalState = create_dynamic_enum("EmtionalState", list(emotion_descriptions.keys()))

Sentiment = create_dynamic_enum("Sentiment", ["positive", "neutral", "negative"])

In [41]:
class CharacterRelationState(BaseModel):
    """1-directional Relationship state between 2 characters"""
    character_uid: str
    emotion: EmotionalState
    knowledge: List[str]
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

class CharacterState(BaseModel):
    sentiment: Sentiment
    emotion: EmotionalState
    social_relations: List[CharacterRelationState]
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True
    

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/3577680599.py:8: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid
/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/3577680599.py:17: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [42]:
CHARACTER_STATE_INIT_INSTRUCTION = '''You are a storyteller which is coming up with an initial state of 2 characters
# Given Informations
You will be given the following informations
* 2 Character descriptions
* Scene description
Write the initial state of each character at the start of the scene

## Characters
Character1:
{character1}

Character2:
{character2}

## Scene
{scene}

# Character State Initialization:
A character's state consists of the following informations
* polarity: sentiment towards the character (positive/neutral/negative)
* emotion: emotional state of the character
* social_relations: a one-directional relationship with other characters. (ex. what character1 thinks of character2)
    * character_uid: uid of the target character
    * emotion: emotional relation towards the other character
    * knowledge: what the character knows about the other character 
* only write the relations between the given characters

## Emotions
utilize the following emotion descriptions (from Plutchik's emotion wheel) to describe the emotional state of the character
* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance social connections and overall resilience by radiating positivity
* "trust": Trust is the reassuring feeling of confidence and security in another person or situation. It builds from consistent, reliable interactions and underpins strong, supportive relationships. This emotion fosters cooperation and reduces anxiety by creating a sense of safety
* "fear": Fear is an instinctive response to perceived threats that activates our fight-or-flight mechanism. It heightens awareness and prepares our body to respond quickly to danger, making it essential for survival. Despite its discomfort, fear is a crucial signal that prompts protective action and risk assessment
* "surprise": Surprise occurs when we encounter the unexpected, momentarily halting our regular thought process. This emotion can be positive, neutral, or even negative, depending on the context, and often sparks curiosity about what comes next. Its brief nature helps redirect our focus and encourages adaptive responses to new situations
* "sadness": Sadness is a deep, reflective emotion that often emerges from loss, disappointment, or unmet expectations. It can lead to introspection and a desire for support as we navigate feelings of grief or dejection. Although challenging, sadness can also foster empathy and pave the way for emotional healing and growth
* "disgust": Disgust is an aversive emotion that signals rejection toward something perceived as harmful, unclean, or morally offensive. It serves as a protective mechanism, prompting us to avoid substances or situations that might be dangerous. This emotion plays a vital role in maintaining both physical health and ethical boundaries
* "anger": Anger arises when we perceive injustice, frustration, or a threat to our well-being, often urging us to act in response. It can manifest as physical tension and heightened energy, signaling that something in our environment needs to change. When managed effectively, anger can motivate constructive action and help assert personal boundaries
* "anticipation": Anticipation is the forward-looking emotion characterized by a mix of excitement and apprehension about future events. It motivates preparation and planning while balancing hope with cautious vigilance. This emotion bridges the gap between our present state and the potential for positive outcomes in the future

Return in the following json format
{{
    "character1_uid": str,
    "character1_state": {{
        "polarity": str,
        "emotion": str,
        "social_relations": [
            {{
                "character_uid": str,
                "emotion": str,
                "knowledge": List[str]
            }},
            ...
        ]
    }},
    "character1_uid": str,
    "character2_state": {{
        "polarity": str,
        "emotion": str,
        "social_relations": [
            {{
                "character_uid": str,
                "emotion": str,
                "knowledge": List[str]
            }},
            ...
        ]
    }}
}}
* only return the name of the referenced tropes in utilized_reference_tropes

Rules:
* trope must be detailed description in minimum 3 sentences.
* trope must focus on character behavioral, emotional interaction and less about the surrounding settings (environment) of the characters
* trope should describe a single small scene, keep the scope small
* mention the character's gender, age, personality, living condition that are only relevant to the trope
'''

In [43]:
class CharacterStateInitOutput(BaseModel):
    character1_uid: str
    character1: CharacterState
    
    character2_uid: str
    character2: CharacterState
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_89360/3104826172.py:9: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [44]:
state_init_instruction = CHARACTER_STATE_INIT_INSTRUCTION.format(
    character1=character1.model_dump_json(),
    character2=character2.model_dump_json(),
    scene=initialized_scene.model_dump_json(),
)
# print(refine_instruction)

In [45]:
messages = [
    {"role": "user", "content": state_init_instruction}
]
decode_params = {"temperature": 0.95}

response = openai_client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=CharacterStateInitOutput,
    **decode_params,
)
initialzed_character_states = response.choices[0].message.parsed

In [46]:
print(initialzed_character_states.model_dump_json(indent=4))

{
    "character1_uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "character1": {
        "sentiment": "neutral",
        "emotion": "anticipation",
        "social_relations": [
            {
                "character_uid": "00d66087-9b3b-46da-bd74-bf45cbe81d3c",
                "emotion": "trust",
                "knowledge": [
                    "Vivienne LaRoux is a renowned style influencer with an assertive demeanor.",
                    "Vivienne initially greets Zephyr with icy indifference during their interaction at the gala.",
                    "Despite Vivienne's dismissive nature, Zephyr sees potential for a deeper connection."
                ]
            }
        ]
    },
    "character2_uid": "00d66087-9b3b-46da-bd74-bf45cbe81d3c",
    "character2": {
        "sentiment": "neutral",
        "emotion": "surprise",
        "social_relations": [
            {
                "character_uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
                "emotion": "surpr

In [47]:
scene_id = str(uuid.uuid4())

scene_dict = {
    "uid": scene_id,
    "stage": 0,
    "scene": initialized_scene.model_dump(),
    "character_states": initialzed_character_states.model_dump(),
}
with open(f"simschat/scene/{scene_id}.json", "w") as f:
    f.write(json.dumps(scene_dict, indent=4))

In [48]:
print(scene_id)

2f2462c1-bdef-4089-b206-c47decd841f3
